In [1]:
pip install scx>=1.2.1 #Note: This may throw an Error in Google Colab since Colab uses an outdated duckdb package. This file should still work in Colab without issues.

Note: you may need to restart the kernel to use updated packages.


<p class="MsoNormal">Before deciding which hub to rent, Food on Wheels would like to add one more requirement to the model.</p>
<p class="MsoNormal">Food on Wheels wants to ensure that at least 75% of the demand is served in less than 7 minutes, from the hubs to all city areas.</p>
<p class="MsoNormal">Adjust your model from Part 3 to help the company decide which hub to rent (only one hub) to <strong>minimize the total delivery time per day while also complying with the new LOS policy. </strong>Note that you can still use the existing hubs (Hub 1, Hub 2, Hub 3) in the model. <o:p></o:p></p>
<p></p>
<a href="https://youtu.be/4jiQQhb2n3Q">FOW P4 Video Walkthrough Link</a><br/>
<iframe width="560" height="315"
    src="https://www.youtube.com/embed/4jiQQhb2n3Q" frameborder="0" allowfullscreen>
</iframe>

In [2]:
from scx.optimize import Model

In [3]:
transport = [
    # Central Kitchen to Owned Hubs
    {
        'origin_name':'K1',
        'destination_name':'H1',
        'time': 2,
    },
    {
        'origin_name':'K1',
        'destination_name':'H2',
        'time': 5,
    },
    {
        'origin_name':'K1',
        'destination_name':'H3',
        'time': 7,
    },
    # Central Kitchen to Hubs for Hire
    {
        'origin_name':'K1',
        'destination_name':'H4',
        'time': 5,
    },
    {
        'origin_name':'K1',
        'destination_name':'H5',
        'time': 6,
    },
    {
        'origin_name':'K1',
        'destination_name':'H6',
        'time': 4,
    },
    # Owned Hubs to Areas
    {
        'origin_name':'H1',
        'destination_name':'A1',
        'time': 13,
    },
    {
        'origin_name':'H1',
        'destination_name':'A2',
        'time': 7,
    },
    {
        'origin_name':'H1',
        'destination_name':'A3',
        'time': 14,
    },
    {
        'origin_name':'H1',
        'destination_name':'A4',
        'time': 14,
    },
    {
        'origin_name':'H1',
        'destination_name':'A5',
        'time': 5,
    },
    {
        'origin_name':'H2',
        'destination_name':'A1',
        'time': 15,
    },
    {
        'origin_name':'H2',
        'destination_name':'A2',
        'time': 4,
    },
    {
        'origin_name':'H2',
        'destination_name':'A3',
        'time': 10,
    },
    {
        'origin_name':'H2',
        'destination_name':'A4',
        'time': 8,
    },
    {
        'origin_name':'H2',
        'destination_name':'A5',
        'time': 7,
    },
    {
        'origin_name':'H3',
        'destination_name':'A1',
        'time': 3,
    },
    {
        'origin_name':'H3',
        'destination_name':'A2',
        'time': 3,
    },
    {
        'origin_name':'H3',
        'destination_name':'A3',
        'time': 11,
    },
    {
        'origin_name':'H3',
        'destination_name':'A4',
        'time': 11,
    },
    {
        'origin_name':'H3',
        'destination_name':'A5',
        'time': 10,
    },
    # Hubs for Hire to Areas
    {
        'origin_name':'H4',
        'destination_name':'A1',
        'time': 10,
    },
    {
        'origin_name':'H4',
        'destination_name':'A2',
        'time': 13,
    },
    {
        'origin_name':'H4',
        'destination_name':'A3',
        'time': 14,
    },
    {
        'origin_name':'H4',
        'destination_name':'A4',
        'time': 8,
    },
    {
        'origin_name':'H4',
        'destination_name':'A5',
        'time': 12,
    },
    {
        'origin_name':'H5',
        'destination_name':'A1',
        'time': 5,
    },
    {
        'origin_name':'H5',
        'destination_name':'A2',
        'time': 8,
    },
    {
        'origin_name':'H5',
        'destination_name':'A3',
        'time': 4,
    },
    {
        'origin_name':'H5',
        'destination_name':'A4',
        'time': 9,
    },
    {
        'origin_name':'H5',
        'destination_name':'A5',
        'time': 11,
    },
    {
        'origin_name':'H6',
        'destination_name':'A1',
        'time': 13,
    },
    {
        'origin_name':'H6',
        'destination_name':'A2',
        'time': 5,
    },
    {
        'origin_name':'H6',
        'destination_name':'A3',
        'time': 8,
    },
    {
        'origin_name':'H6',
        'destination_name':'A4',
        'time': 7,
    },
    {
        'origin_name':'H6',
        'destination_name':'A5',
        'time': 10,
    },

]

kitchens = [
    {
        'name': 'K1',
    },
]

hubs = [
    {
        'name': "H1",
        'limit': 150,
        'type': 'owned'
    },
    {
        'name': "H2",
        'limit': 100,
        'type': 'owned'
    },
    {
        'name': "H3",
        'limit': 150,
        'type': 'owned'
    },
    {
        'name': "H4",
        'limit': 200,
        'type': 'rent'
    },
    {
        'name': "H5",
        'limit': 200,
        'type': 'rent'
    },
    {
        'name': "H6",
        'limit': 200,
        'type': 'rent'
    },
]



demand = [
    {
        'name':'A1',
        'demand':101
    },
    {
        'name':'A2',
        'demand':61
    },
    {
        'name':'A3',
        'demand':101
    },
    {
        'name':'A4',
        'demand':87
    },
    {
        'name':'A5',
        'demand':50
    },
]

In [4]:
for t in transport:
    # Create decision variables for each item in transport
    t['amt']=Model.variable(name=f"{t['origin_name']}__{t['destination_name']}__amt", lowBound=0)

In [5]:
for h in hubs:
    h['use']=Model.variable(name=f"{h['name']}__use", cat="Binary")

In [6]:
M=999999

In [7]:
# Initialize the model
my_model = Model(name="FoW", sense='minimize')


# Add the Objective Fn
my_model.add_objective(
    fn=Model.sum([t['amt']*t['time'] for t in transport])
)

# Add Constraints
## Demand Constraint
for d in demand:
    my_model.add_constraint(
        name=f"{d['name']}__demand",
        fn=Model.sum([t['amt'] for t in transport if t['destination_name']==d['name']]) >= d['demand']
    )

## Hub Constraints
for h in hubs:
    ## Hub Capacity Constraint
    my_model.add_constraint(
        name=f"{h['name']}__capacity",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==h['name']]) <= h['limit']
    )

    ## Conservation of Flow Constraint
    my_model.add_constraint(
        name=f"{h['name']}__conservation_of_flow",
        # Set inbound flows for the hub equal to outbound flows
        fn=Model.sum(
            [t['amt'] for t in transport if t['destination_name']==h['name']]
        ) == Model.sum(
            [t['amt'] for t in transport if t['origin_name']==h['name']]
        )
    )

    ## Hub linking constraint
    my_model.add_constraint(
        name=f"{h['name']}__linking",
        fn=Model.sum([t['amt'] for t in transport if t['origin_name']==h['name']]) <= h['use']*M
    )

## Only allow one rental hub to be chosen
my_model.add_constraint(
    name=f"only_one_rental_hub",
    fn=Model.sum([h['use'] for h in hubs if h['type']=='rent']) == 1
)

## 75% Demand Served in under 7 minutes from hubs
demand_names = [d['name'] for d in demand]
my_model.add_constraint(
    name=f"75_pct_demand_under_7_minutes",
    fn=Model.sum([t['amt'] for t in transport if t['time']<7 and t['destination_name'] in demand_names]) >= Model.sum([d['demand'] for d in demand])*.75
)

# Solve the model
my_model.solve()

In [8]:
my_model.show_formulation()

FoW:
MINIMIZE
13*H1__A1__amt + 7*H1__A2__amt + 14*H1__A3__amt + 14*H1__A4__amt + 5*H1__A5__amt + 15*H2__A1__amt + 4*H2__A2__amt + 10*H2__A3__amt + 8*H2__A4__amt + 7*H2__A5__amt + 3*H3__A1__amt + 3*H3__A2__amt + 11*H3__A3__amt + 11*H3__A4__amt + 10*H3__A5__amt + 10*H4__A1__amt + 13*H4__A2__amt + 14*H4__A3__amt + 8*H4__A4__amt + 12*H4__A5__amt + 5*H5__A1__amt + 8*H5__A2__amt + 4*H5__A3__amt + 9*H5__A4__amt + 11*H5__A5__amt + 13*H6__A1__amt + 5*H6__A2__amt + 8*H6__A3__amt + 7*H6__A4__amt + 10*H6__A5__amt + 2*K1__H1__amt + 5*K1__H2__amt + 7*K1__H3__amt + 5*K1__H4__amt + 6*K1__H5__amt + 4*K1__H6__amt + 0
SUBJECT TO
A1__demand: H1__A1__amt + H2__A1__amt + H3__A1__amt + H4__A1__amt
 + H5__A1__amt + H6__A1__amt >= 101

A2__demand: H1__A2__amt + H2__A2__amt + H3__A2__amt + H4__A2__amt
 + H5__A2__amt + H6__A2__amt >= 61

A3__demand: H1__A3__amt + H2__A3__amt + H3__A3__amt + H4__A3__amt
 + H5__A3__amt + H6__A3__amt >= 101

A4__demand: H1__A4__amt + H2__A4__amt + H3__A4__amt + H4__A4__amt
 + H5__A

In [9]:
# Show the outputs
my_model.show_outputs()

{'objective': 4085.0,
 'status': 'Optimal',
 'variables': {'H1__A1__amt': 0.0,
               'H1__A2__amt': 13.0,
               'H1__A3__amt': 0.0,
               'H1__A4__amt': 0.0,
               'H1__A5__amt': 50.0,
               'H1__use': 1.0,
               'H2__A1__amt': 0.0,
               'H2__A2__amt': 13.0,
               'H2__A3__amt': 0.0,
               'H2__A4__amt': 87.0,
               'H2__A5__amt': 0.0,
               'H2__use': 1.0,
               'H3__A1__amt': 101.0,
               'H3__A2__amt': 35.0,
               'H3__A3__amt': 0.0,
               'H3__A4__amt': 0.0,
               'H3__A5__amt': 0.0,
               'H3__use': 1.0,
               'H4__A1__amt': 0.0,
               'H4__A2__amt': 0.0,
               'H4__A3__amt': 0.0,
               'H4__A4__amt': 0.0,
               'H4__A5__amt': 0.0,
               'H4__use': 0.0,
               'H5__A1__amt': 0.0,
               'H5__A2__amt': 0.0,
               'H5__A3__amt': 101.0,
               'H5

In [10]:
# Show the amounts of items that have flowed through hub 5
print(Model.sum(
    [t['amt'].value() for t in transport if t['origin_name']=='H5']
))

101.0
